# Loading data

In [ ]:
import numpy as np
import osyris

## Loading a Ramses output

We load a data output from a star formation simulation

In [ ]:
path = "osyrisdata/starformation"
data = osyris.Dataset(8, path=path).load()

and show a 2D histogram of the gas density versus the magnetic field strength

In [ ]:
osyris.histogram2d(
    data["hydro"]["density"], data["hydro"]["B_field"], norm="log", loglog=True
)

## Selective loading

### Filtering on cell values

It is possible to load only a subset of the cells, by using custom functions to perform the selection.

As an example, to load all the cells with $\rho > 10^{-15}~{\rm g~cm}^{-3}$, we use a selection criterion

In [ ]:
data = osyris.Dataset(8, path=path).load(
    select={"hydro": {"density": lambda d: d > 1.0e-15 * osyris.units("g/cm**3")}}
)

and we can now see in the resulting histogram that all the low-density cells have been left out:

In [ ]:
osyris.histogram2d(
    data["hydro"]["density"], data["hydro"]["B_field"], norm="log", loglog=True
)

Multiple selection criteria are ANDed:

In [ ]:
data = osyris.Dataset(8, path=path).load(
    select={
        "hydro": {"density": lambda d: d > 1.0e-16 * osyris.units("g/cm**3")},
        "amr": {"x": lambda x: x > 1500.0 * osyris.units("au")},
    }
)

In [ ]:
osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},
    dx=1000 * osyris.units("au"),
    origin=data["amr"]["position"][np.argmax(data["hydro"]["density"])],
    direction="z",
)

### Only loading certain groups or variables

It is also possible to select which groups to load.
The different groups are `'amr'`, `'hydro'`, `'grav'`, `'rt'`, and `'part'`.
For example, to load only the `amr` and `hydro` groups, use

In [ ]:
data = osyris.Dataset(8, path=path).load(["amr", "hydro"])
data.keys()

It is also possible to load or skip only a single variable in a group,
by setting the key to `True` or `False` in the `select` dict.

The convention is:

- If at least one key in a group is set to `True`, all unspecified variables in the group are skipped
- If a key is set to `False` in a group, all other variables are loaded

In [ ]:
data = osyris.Dataset(8, path=path).load(select={"hydro": {"density": False}})
"density" in data["hydro"]

In [ ]:
data = osyris.Dataset(8, path=path).load(
    select={"hydro": {"temperature": True, "thermal_pressure": True}}
)
data["hydro"].keys()

### Selecting AMR levels

Selecting AMR levels uses the same syntax as selecting other variables,
but works slightly differently under the hood.
A maximum level will be found by testing the selection function provided,
and only the required levels will be traversed by the loader,
thus speeding up the loading process.

Hence,

In [ ]:
data = osyris.Dataset(8, path=path).load(select={"amr": {"level": lambda l: l < 7}})
data["amr"]["level"]

will only read levels 1 to 6, while

In [ ]:
data = osyris.Dataset(8, path=path).load(
    select={"amr": {"level": lambda l: np.logical_and(l > 5, l < 9)}}
)
data["amr"]["level"]

will read levels 1 to 8, but will then discard all cells with `level` < 6.

### Loading only selected CPU outputs

It is also possible to feed a list of CPU numbers to read from.

In [ ]:
data = osyris.Dataset(8, path=path).load(cpu_list=[1, 2, 10, 4, 5])

In [ ]:
osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},
    dx=2000 * osyris.units("au"),
    origin=data["amr"]["position"][np.argmax(data["hydro"]["density"])],
    direction="z",
)

<div class="alert alert-info">

**Note**

When performing a selection (using `select`) on spatial position `x`, `y`, or `z`,
and if the ordering of the cells in the AMR mesh is using the Hilbert curve,
then a pre-selection is automatically made on the CPU files to load to speed-up the loading.

</div>

## Example: loading a region around a sink particle

Combining some of the methods illustrated above,
we show here how to load only a small region 400 AU wide around a sink particle.

We begin by loading only the sink particle data, by using `select='sink'` in the `load` method:

In [ ]:
data = osyris.Dataset(8, path=path).load(select="sink")
data

The `sink` data group contains the positions of the sink particles.
We wish to load the region around the first sink particle, which will be the center of our domain.

In [ ]:
center = data["sink"]["position"][0]
dx = 200 * osyris.units("au")

We are now in a position to load the rest of the data (`amr`, `hydro`, etc.)
and perform a spatial selection based on the sink's position
(note how the loading below is only looking through 6 files, as opposed to 12 at the top of the notebook,
because it uses the knowledge from the Hilbert space-filling curve to skip CPUs that are not connected
to the domain of interest).

In [ ]:
data.load(
    select={
        "amr": {
            "x": lambda x: (x > center.x - dx) & (x < center.x + dx),
            "y": lambda y: (y > center.y - dx) & (y < center.y + dx),
            "z": lambda z: (z > center.z - dx) & (z < center.z + dx),
        }
    }
)

Finally, we can make a map of the gas density around our sink particle
(note that no spatial limits are specified in the `map` call,
it is making a map using all the cells that have been loaded)

In [ ]:
osyris.map(
    data["hydro"]["density"],
    {
        "data": data["sink"]["position"],
        "mode": "scatter",
        "c": "white",
        "s": 20.0 * osyris.units("au"),
        "alpha": 0.7,
    },
    norm="log",
    direction="z",
    origin=center,
)